In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY_1')

def get_open_ai_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    open_ai_response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )
    return open_ai_response.choices[0].message["content"]

In [3]:
import requests

perplexity_api_key = os.getenv('PERPLEXITY_API_KEY_1')

purl = "https://api.perplexity.ai/chat/completions"
pheaders = {
    "accept": "application/json",
    "authorization": f'Bearer {perplexity_api_key}',
    "Content-Type": "application/json"
}

def get_perplexity_completion(prompt, model="llama-2-70b-chat"):
    messages = [{"role": "user", "content": prompt}]
    data = {
        "messages": messages,
        "model": model,
        "temperature": 0.2
    }
    perplexity_response = requests.post(purl, headers=pheaders, json=data)
    return perplexity_response.json()["choices"][0]["message"]["content"]

In [4]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY_1')
anthropic = Anthropic(api_key=anthropic_api_key)

def get_anthropic_completion(prompt, model="claude-2.1"):
    completion = anthropic.completions.create(
        model=model,
        max_tokens_to_sample=1000,
        temperature=0.2,
        prompt=f"${HUMAN_PROMPT} ${prompt}${AI_PROMPT}",
    )
    return completion.completion

In [5]:
import anthropic_bedrock
from anthropic_bedrock import AnthropicBedrock

aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')

abclient = AnthropicBedrock(
    # Authenticate by either providing the keys below or use the default AWS credential providers, such as
    # using ~/.aws/credentials or the "AWS_SECRET_ACCESS_KEY" and "AWS_ACCESS_KEY_ID" environment variables.
    aws_access_key=aws_access_key,
    aws_secret_key=aws_secret_key,
    # Temporary credentials can be used with aws_session_token.
    # Read more at https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_temp.html.
    # aws_session_token="<session_token>",
    # aws_region changes the aws region to which the request is made. By default, we read AWS_REGION,
    # and if that's not present, we default to us-east-1. Note that we do not read ~/.aws/config for the region.
    aws_region="us-east-1",
)

def get_bedrock_anthropic_completion(aprompt, amodel="anthropic.claude-instant-v1"):
    completion = abclient.completions.create(
        model=amodel,
        max_tokens_to_sample=1000,
        temperature=0.2,
        prompt=f"{anthropic_bedrock.HUMAN_PROMPT} ${aprompt} {anthropic_bedrock.AI_PROMPT}",
    )
    return completion.completion

In [6]:
prompt = """
You are an assistant that specializes in travel planning. You are communicating with the LLM agent so return the JSON array only! You task is to plan a walking tour in Boston where tourists would like to visit 5 landmark places on the Freedom Trail.

For each place, please provide the following details in a plain English:
- Name;
- A brief description of the place with a bit of its history (about five sentences);
- location (latitude and longitude).

The list should be optimized for a walking tour, meaning the places should be ordered in the way one would visit them walking along the Freedom Trail.

The information should be provided in the following valid, JSON format:

{
  "place_id": "<id>",
  "name": "<name>",
  "description": "<description>",
  "location": "<latitude,longitude>"
}

Make sure you return the only json as a result, no comments, no surrounding text, json array only!
"""

In [10]:
def print_values(name, model, running_time, response):
    print(f"======={name} API ({model}) Begins========")
    print(f"running_time=>{running_time:.2f}")
    print(response)
    print(f"======={name} API ({model}) Ends========")

In [11]:
import time

start = time.time()
model = "gpt-3.5-turbo"
response = get_open_ai_completion(prompt, model)
running_time = time.time() - start
print_values("OpenAI", model, running_time, response)

models = ["mistral-7b-instruct", "pplx-70b-chat"]
for model in models:
    start = time.time()
    response = get_perplexity_completion(prompt, model)
    running_time = time.time() - start
    print_values("Perplexity", model, running_time, response)

model = "claude-instant-1.2"
start = time.time()
response = get_anthropic_completion(prompt, model)
running_time = time.time() - start
print_values("Anthropic", model, running_time, response)

model = 'anthropic.claude-instant-v1'
start = time.time()
response = get_bedrock_anthropic_completion(prompt, model)  #claude-instant-1.2;claude-2.0
running_time = time.time() - start
print_values("Anthropic Bedrock", model, running_time, response)

=======OpenAI API (gpt-3.5-turbo) Begins========
running_time=>12.75
[
  {
    "place_id": "1",
    "name": "Massachusetts State House",
    "description": "The Massachusetts State House is the state capitol and seat of government for the Commonwealth of Massachusetts. Built in 1798, it is a prominent landmark on Beacon Hill and is known for its distinctive gold dome. The building houses the Massachusetts General Court, the Governor's offices, and various other government agencies. The State House is open to the public and offers guided tours. Its location is 42.3589° N, 71.0636° W.",
    "location": "42.3589,-71.0636"
  },
  {
    "place_id": "2",
    "name": "Park Street Church",
    "description": "Park Street Church is a historic Congregational church located near the Boston Common. Established in 1809, it is one of the oldest churches in Boston. The church played a significant role in the abolitionist movement and was a center for social reform in the 19th century. Its location is